### Árboles de decisión
#### Machine Learning I - Maestría en Analítica Aplicada
#### Universidad de la Sabana
#### Prof: Hugo Franco
#### Ejemplo: predicción de desarrollo de la diabetes (población de la India)

Se usarán los métodos de gestión de cadenas del módulo __six__ y la biblioteca __pydotplus__ (interfaz con __graphviz__)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn import metrics 

# Para representar el árbol de decisión
from six import StringIO  # Capa de compatibilidad entre Python 2 y Python 3
from sklearn.tree import export_graphviz # Visualización de árboles de decisión
from IPython.display import Image # Módulo de gestión de imágenes digitales 
import pydotplus # Envoltorio mejorado del lenguaje "dot" de graphviz 

##### Etiquetas de los datos a partir del conocimiento del problema

El Dataset de ejemplo proviene del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales de la India. La idea es predecir si un paciente puede o no tener _diabetes mellitus_, con base en datos de tamizaje como la edad, el número de embarazos, los niveles de glucosa, insulina y presión arterial, el grosor de la piel y la "función de pedigree" (asociada a factores genéticos).
https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv


In [ ]:
# Se define el nombre de las columnas en una lista 
nombre_columnas = ['Embarazos', 'Glucosa', 'Presion Arterial', 'Grosor Piel', 'Insulina', 'Indice Masa Corporal', 'Función de Pedigree', 'Edad', 'Etiqueta']

# Se usa la biblioteca Pandas para cargar el dataset desde un archivo de texto 
# "separado por comas" (CSV), usando los nombres de columna previamente definidos
dataset = pd.read_csv("diabetes.csv", header=1, names=nombre_columnas)

In [ ]:
dataset.head(10)

In [ ]:
feature_cols = ['Embarazos', 'Insulina', 'Indice Masa Corporal', 'Edad', 'Glucosa', 'Presion Arterial', 'Función de Pedigree']

X = dataset[feature_cols] # Características
y = dataset.Etiqueta # Variable objetivo

Se divide el dataset entre los conjuntos de entrenamiento y prueba:


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% para entrenamiento
print("Tamaño de X_train: {}\nTamaño de y_train: {}".format(X_train.shape, y_train.shape)) #Salida con formato en Python
print("Tamaño de X_test: {}\nTamaño de y_test: {}".format(X_test.shape, y_test.shape))

#### Creación y entrenamiento del modelo
Se crea un objeto de clase _DecisionTreeClassifier_ y se usa el dataset de entrenamiento para el _aprendizaje_ del árbol de decisión. El árbol entrenado permite obtener las salidas _reales_ del modelo, que serán contrastadas con las salidas _esperadas_ según lo contenido en el dataset original.

In [ ]:
dtree = DecisionTreeClassifier()

# Entrenamiento
dtree = dtree.fit(X_train,y_train)

#Predicción para evaluación
y_pred = dtree.predict(X_test)

Se concatenan las columnas de caraterísticas para prueba (_X_test_), valor esperado de registros de prueba (_y_test_) y valores entregados (predichos) por el modelo para los registros de prueba (_y_pred_). Este último requiere una conversión a series de datos de Pandas a partir del formato de salida del método _dtree.predict_.

In [ ]:
pd.concat([X_test, y_test, pd.Series(y_pred, name='Predicción', index=X_test.index)], 
          ignore_index=False, axis=1)

Se imprimen los resultados de desempeño obtenidos mediante _validación cruzada_

In [ ]:
cv_scores = cross_val_score(dtree, X_train, y_train, cv=5)

# Escribe en pantalla los desempeños ("accuracy") para cada "pliegue" (fold)
print (cv_scores);

# Escribe en pantalla el promedio de la métrica "accuracy" en el entrenamiento 
print(np.average(cv_scores))

# Escribe en pantalla la métrica "accuracy" para prueba (datos no usados en el entrenamiento)
print("Exactitud (accuracy) en prueba (testing):",metrics.accuracy_score(y_test, y_pred))

__Ejemplo:__ acceder a un registro arbitrario del dataset y ver su clasificación

In [ ]:
print(X_test.iloc[210])
print(y_test.iloc[210])


__Ejemplo:__ crear un nuevo caso (similar a una situación real) para obtener su clasificación con el modelo entrenado

In [ ]:
data = {'Embarazos':[2],'Insulina':[120.000],'Indice Masa Corporal':[20],'Edad':[32.000],'Glucosa':[190],'Presion Arterial':[100],'Función de Pedigree':[0.75]}
query=pd.DataFrame(data)

nueva_clasificacion=dtree.predict(query)

print("Clase: ",nueva_clasificacion)

La curva ROC (Receiver-Operating Characteristic) da una idea de la calidad del clasificador: entre más se acerque a la esquina superior izquierda, mayor será el área bajo la curva.

In [ ]:
fpr,tpr,_r=metrics.roc_curve(y_test,y_pred)
roc_auc = metrics.auc(fpr, tpr)
print(roc_auc)
metrics.plot_roc_curve(dtree, X_test, y_test)

#### Visualización del modelo obtenido
El siguiente segmento de código grafica el árbol de decisión obtenido

In [ ]:
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
               filled=True, rounded=True,
               special_characters=True, feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())